## The CIFAR-10 Dataset
CIFAR-10 consists of 60000 32x32 color images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images. Here are the classes in the dataset, as well as 10 random images from each:

In [54]:
from keras.datasets import cifar10
from keras.utils import np_utils

(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# convert class vectors to binary vectors
Y_train = np_utils.to_categorical(y_train)
Y_test = np_utils.to_categorical(y_test)

print('X_train shape:', X_train.shape)
print('Y_train shape:', Y_train.shape)
print('X_test shape:', X_test.shape)
print('Y_test shape:', Y_test.shape)

X_train shape: (50000, 32, 32, 3)
Y_train shape: (50000, 10)
X_test shape: (10000, 32, 32, 3)
Y_test shape: (10000, 10)


In [55]:
# normalize inputs from 0-255 to 0.0-1.0
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0

print('X_train shape:', X_train.shape)
print('X_val shape:', X_test.shape)

X_train shape: (50000, 32, 32, 3)
X_val shape: (10000, 32, 32, 3)


In [49]:
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers import Activation
from keras.callbacks import EarlyStopping

img_size = 32
img_channels = 3
nb_classes = 10
batch_size = 128
epoch_max = 15
early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0)

In [50]:
def evaluate(model, hist, plt_path):
    score = model.evaluate(X_test_gray, Y_test, verbose=0)
    print('Test loss: %.3f' % score[0])
    print('Test accuracy: %.3f' % score[1])
    plot_validation_history(hist, plt_path)

In [51]:
def fit(model):
    hist = model.fit(X_train, Y_train, \
                    batch_size=batch_size, \
                    nb_epoch=epoch_max, \
                    validation_data=(X_val_gray, Y_val), \
                    callbacks=[early_stop], \
                    shuffle=True, verbose=0)
    return hist

In [52]:
from keras.layers.convolutional import Convolution2D, MaxPooling2D

model_cnn = Sequential()

# conolutional hidden layers
for i in range(6):
    model_cnn.add(Convolution2D(32, 3, 3, 
                        input_shape=(img_size, img_size, img_channels), 
                        border_mode='same', activation='relu'))
    if (i + 1) % 2 == 0:
        model_cnn.add(MaxPooling2D(pool_size=(2, 2), border_mode='same'))
    
print('Output shape of last convolution layers: {0}'.format(model_cnn.output_shape))
model_cnn.add(Flatten())

# fully connected hidden layers
for i in range(2):
    model_cnn.add(Dense(512))
    model_cnn.add(Activation('relu'))
    
# output layer
model_cnn.add(Dense(nb_classes, activation='softmax'))

# compile model
model_cnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model_cnn.summary()

C:\Users\niall\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(32, 32, 3..., activation="relu", padding="same")`
  if __name__ == '__main__':
C:\Users\niall\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), padding="same")`
  # This is added back by InteractiveShellApp.init_path()


Output shape of last convolution layers: (None, 4, 4, 32)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_91 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_92 (Conv2D)           (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_46 (MaxPooling (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_93 (Conv2D)           (None, 16, 16, 32)        9248      
_________________________________________________________________
conv2d_94 (Conv2D)           (None, 16, 16, 32)        9248      
_________________________________________________________________
max_pooling2d_47 (MaxPooling (None, 8, 8, 32)          0         
_________________________________________________________________
conv2d_95 (Conv2D)

In [56]:
# Training
model_cnn.fit(x=X_train,y=Y_train, epochs=10)

Epoch 1/10
50000/50000 [==============================] - 311s 6ms/step - loss: 1.5563 - acc: 0.4206
Epoch 2/10
50000/50000 [==============================] - 290s 6ms/step - loss: 1.1259 - acc: 0.5960
Epoch 3/10
50000/50000 [==============================] - 316s 6ms/step - loss: 0.9243 - acc: 0.6713
Epoch 4/10
50000/50000 [==============================] - 292s 6ms/step - loss: 0.8025 - acc: 0.7169
Epoch 5/10
50000/50000 [==============================] - 294s 6ms/step - loss: 0.7095 - acc: 0.7505
Epoch 6/10
50000/50000 [==============================] - 299s 6ms/step - loss: 0.6313 - acc: 0.7766
Epoch 7/10
50000/50000 [==============================] - 290s 6ms/step - loss: 0.5621 - acc: 0.8005 1s - loss: 0.5623 - acc:
Epoch 8/10
50000/50000 [==============================] - 288s 6ms/step - loss: 0.4947 - acc: 0.8232
Epoch 9/10
50000/50000 [==============================] - 301s 6ms/step - loss: 0.4360 - acc: 0.8443
Epoch 10/10
50000/50000 [==============================] - 289s 6m

In [58]:
scores = model_cnn.evaluate(X_test, Y_test, verbose = 0)
print("Error percentage: %.2f%%" %(100 - scores[1] * 100))

Error percentage: 27.53%
